# Data Quality Quick Viz (ADR trend + Intraday VWAP)

This notebook helps you visually inspect your Dukascopy-derived Parquet library.
- **ADR trend**: median true range over time per symbol/timeframe
- **Intraday VWAP**: pick a day and overlay VWAP vs. price to spot anomalies

**Instructions**:
1. Set `ROOT`, `TF`, and `SYMBOL` below, or let the notebook auto-pick the first available file.
2. Run all cells.
3. If you don't have 5m data, it will try 1h.


In [ ]:
import pandas as pd, numpy as np, pathlib, random
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ROOT = 'data'           # Parquet root
TF = None               # e.g., '5m' or '1h'; if None, auto-detect
SYMBOL = None           # e.g., 'EURUSD'; if None, auto-pick first file

root = pathlib.Path(ROOT)
candidates = []
for folder in ['prices_5m','prices_1h','prices_1d','prices_1m']:
    p = root / folder
    if p.exists():
        for f in sorted(p.glob('*.parquet')):
            candidates.append((folder.split('_')[1], f))

if not candidates:
    raise SystemExit("No Parquet files found under data/prices_* . Ingest first.")

if SYMBOL is not None:
    candidates = [(tf, f) for tf, f in candidates if f.stem.upper()==SYMBOL.upper()]
    if not candidates:
        raise SystemExit(f"No Parquet found for symbol {SYMBOL}")

if TF is not None:
    candidates = [(tf, f) for tf, f in candidates if tf==TF]
    if not candidates:
        raise SystemExit(f"No Parquet found for timeframe {TF}")

TF, FILE = candidates[0]
SYMBOL = FILE.stem.upper()
TF, FILE, SYMBOL

In [ ]:
df = pd.read_parquet(FILE)
df = df.sort_index()
if df.index.tz is None:
    df.index = df.index.tz_localize('UTC')

print(SYMBOL, TF, len(df), df.index.min(), df.index.max())
df.head()

## ADR Trend (median |High-Low|)
Use a 20-period rolling median ADR to visualize range changes.

In [ ]:
if TF in ['1d','1h','5m','1m']:
    adr_series = (df['High'] - df['Low']).abs()
    adr_med = adr_series.rolling(20, min_periods=5).median()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=adr_med.index, y=adr_med, name='Rolling median ADR (20)'))
    fig.update_layout(title=f'{SYMBOL} {TF} — Rolling median ADR (20)', xaxis_title='Time (UTC)', yaxis_title='ADR')
    fig.show()
else:
    print('Unsupported TF for ADR plot')

## Intraday VWAP vs Price (single day)
Pick a day with enough bars and overlay VWAP and price to spot anomalies.
For best fidelity, use 1m/5m data. If using 1h, VWAP is coarse but still illustrative.

In [ ]:
def pick_intraday_day(df: pd.DataFrame):
    days = pd.to_datetime(df.index.date).astype('datetime64[ns]').astype('datetime64[ns, UTC]').unique()
    # Prefer a mid-week day to avoid weekend effects
    days = [pd.Timestamp(d).tz_localize('UTC') for d in days]
    days = [d for d in days if d.weekday() < 5]
    return random.choice(days) if days else None

def intraday_slice(df, day):
    day_end = day + pd.Timedelta(days=1)
    return df.loc[(df.index>=day) & (df.index<day_end)]

if TF in ['1m','5m','1h']:
    day = pick_intraday_day(df)
    if day is None:
        raise SystemExit('No valid weekday found in data')
    d = intraday_slice(df, day)
    # VWAP intraday
    pv = (d['Close']*d['Volume']).cumsum()
    vv = d['Volume'].cumsum().replace(0, np.nan)
    vwap = pv / vv

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Candlestick(x=d.index, open=d['Open'], high=d['High'], low=d['Low'], close=d['Close'], name='Price'), secondary_y=False)
    fig.add_trace(go.Scatter(x=d.index, y=vwap, name='VWAP', mode='lines'), secondary_y=False)
    fig.update_layout(title=f'{SYMBOL} {TF} — Intraday VWAP vs Price — {day.date()}', xaxis_title='UTC time')
    fig.show()
else:
    print('Intraday VWAP plot requires 1m/5m/1h data')